In [1]:
import numpy as np
import tensorflow as tf

C:\Users\u69205\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#  LSTM Parameters

In [2]:
batchSize = 24
lstmUnits = 64
numClasses = 2
numDimensions = 300
maxSeqLength=250

In [3]:
wordsList = np.load('wordsList.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')

In [4]:
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [5]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models\pretrained_lstm.ckpt-59999


# Helper Functions

In [6]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    
    if len(split) > 250:
        split = split[0:250]
    
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix

# New Data

In [7]:
inputText = "That movie was terrible."
#inputText = "A great movie"
inputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
print('Positive Sentiment' if predictedSentiment[0] > predictedSentiment[1] else 'Negative Sentiment')

Negative Sentiment


In [8]:
#Ex 2
inputText = "Elon Musk, Ray Kurzweil and others outline the power of big data and existing computer systems, and describe how such technologies have already manipulated world events, particularly recent elections. Worse yet are predictions that robots will displace millions of workers, and that autonomous weapons could wage unsupervised war. Though the film is heavier on summaries than specifics, its messages are troubling nonetheless."
inputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
print('Positive Sentiment' if predictedSentiment[0] > predictedSentiment[1] else 'Negative Sentiment')

Negative Sentiment


In [9]:
#https://www.nytimes.com/2018/08/16/movies/do-you-trust-this-computer-review.html?rref=collection%2Fcollection%2Fmovie-guide&action=click&contentCollection=undefined&region=stream&module=stream_unit&version=latest-stories&contentPlacement=8&pgtype=collection
inputText="We’ve opened Pandora’s box: We’ve unleashed forces that we can’t control, and we can’t stop. We’re in the midst of essentially creating a new life form on Earth."
inputText=inputText + " Had that quote come at the start of “Do You Trust This Computer?” you might think it hyperbole. Yet it’s said at the end. By then the remark seems an understatement."
inputText=inputText + " This part-scary, part-spellbinding documentary, directed by Chris Paine (Who Killed the Electric Car?), talks to a gaggle of experts about current artificial intelligence and about what the future appears destined to bring. Their warnings are so dire that you may start peering sideways at your cellphone, wondering what it’s thinking."
inputText=inputText + " Elon Musk, Ray Kurzweil and others outline the power of big data and existing computer systems, and describe how such technologies have already manipulated world events, particularly recent elections. Worse yet are predictions that robots will displace millions of workers, and that autonomous weapons could wage unsupervised war. Though the film is heavier on summaries than specifics, its messages are troubling nonetheless."
inputText=inputText + " To avoid a nonstop scarefest, Mr. Paine throws in a few old movie clips and allows for a bit of optimism. But the interviewees aren’t budging, and the soundtrack sets a consistently menacing tone."
inputText=inputText + " Slick cuts and glossy images make Do You Trust This Computer? a sleek and engaging watch. Sections on the possibility of artificial consciousness would flabbergast even Philip K. Dick. Still, anxiety runs underneath most of its mind-blowing ideas. We’re told that we may be just a few years away from technologies that are beyond human control. Worry about that if you like, though in the words of one researcher here: I’m not sure it’s going to help."
inputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
print('Positive Sentiment' if predictedSentiment[0] > predictedSentiment[1] else 'Negative Sentiment')

Positive Sentiment


In [10]:
# https://www.nytimes.com/2018/08/23/movies/support-the-girls-review-regina-hall.html?rref=collection%2Fcollection%2Fmovie-guide&action=click&contentCollection=undefined&region=stream&module=stream_unit&version=latest-stories&contentPlacement=2&pgtype=collection
# movie name: Support the girls (genre: Comedy)

inputText = 'Ms. Hall, who appears in almost every scene, is the movie’s animating force. In contrast to Ms. Richardson and Ms. McHayle, who deliver delightful, contrapuntal comic performances, Ms. Hall has to play it fairly straight. Lisa whirls through her day (when she pauses, her body sinks and you feel her fatigue), often with a smile that seems on the verge of breaking. You can see a lifetime in that smile. In moments, as it trembles and almost disappears, and as Ms. Hall’s eyes gently widen and her face briefly freezes, you clearly see both the mask and the woman who wears it. You also see what it takes to be a woman in this man’s world — the spirit, grit, pain and, of course, laughter.'
inputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
print('Positive Sentiment' if predictedSentiment[0] > predictedSentiment[1] else 'Negative Sentiment')

Positive Sentiment


In [11]:
# https://www.nytimes.com/2018/08/16/movies/down-a-dark-hall-review-uma-thurman.html?rref=collection%2Fcollection%2Fmovie-guide
# movie name: down a dark hall (genre: horror)

inputText='When Ms. Thurman saunters onscreen, sporting a smirk to match her loopy accent, it’s as much of a shock as the film’s first spectral scares. With each new element, “Down a Dark Hall” reveals itself, with improbable delight, to be genuinely strange — a movie in which viewers can pick their own pleasure, no two spectators having exactly the same experience.'
nputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
print('Positive Sentiment' if predictedSentiment[0] > predictedSentiment[1] else 'Negative Sentiment')

Positive Sentiment


In [12]:
# https://www.nytimes.com/2018/08/16/movies/do-you-trust-this-computer-review.html?rref=collection%2Fcollection%2Fmovie-guide
# movie name: down a dark hall (genre: horror)

inputText='Elon Musk, Ray Kurzweil and others outline the power of big data and existing computer systems, and describe how such technologies have already manipulated world events, particularly recent elections. Worse yet are predictions that robots will displace millions of workers, and that autonomous weapons could wage unsupervised war. Though the film is heavier on summaries than specifics, its messages are troubling nonetheless.'
nputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
print('Positive Sentiment' if predictedSentiment[0] > predictedSentiment[1] else 'Negative Sentiment')

Positive Sentiment


In [13]:
# https://www.avclub.com/the-20-worst-films-of-2017-1821319277
# film: The House
inputText='Sorry, Chano: Although there’s a germ of a subversive satire somewhere in this not-screened-for-critics item about a couple of anxious suburban squares (Will Ferrell and Amy Poehler) who open an illegal casino in their neighbor’s foreclosed McMansion in order to pay their daughter’s college tuition, it’s lost in the incoherent, lazy direction and writing. The House is everything that’s wrong with the current studio comedy model (“put comedians in a room, roll camera”) packed into a single movie, from the cynically clumsy and hectic pace to the drawn-out, torturous improvisations. The only thing that’s even mildly funny about the film is its shocking nastiness. [Ignatiy Vishnevetsky]'
inputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
print('Positive Sentiment' if predictedSentiment[0] > predictedSentiment[1] else 'Negative Sentiment')

Negative Sentiment


In [14]:
# https://timesofindia.indiatimes.com/entertainment/hindi/movie-reviews/missing/movie-review/63633283.cms
# film: Missing
inputText = 'What good is a thriller that’s not thrilling? It gets worse when you start laughing at a mystery movie. Any and all prospect of this film is lost in the midst of shoddy filmmaking.'
inputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
print('Positive Sentiment' if predictedSentiment[0] > predictedSentiment[1] else 'Negative Sentiment')

Negative Sentiment


# Interactive Mode

In [15]:
inputText = input('Enter your review:')
inputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
print('Positive Sentiment' if predictedSentiment[0] > predictedSentiment[1] else 'Negative Sentiment')

Enter your review:i am bad
Negative Sentiment
